<div style = "text-align: center; color: black; font-family: Times New Roman;">
  <strong><h1>DATA CLEANING<strong></h1>
</div>

<p style="font-family: 'Times New Roman'; font-size: 18px; text-align: center; color: black;">
  For the stock market data, we focused on the three most widely followed U.S. indexes: S&P 500 (<b>^GSPC</b>), NASDAQ (<b>^IXIC</b>), and Dow Jones (<b>^DJI</b>). Using <b>yf.download()</b>, we specified the stock ticker, start date, end date, and a monthly interval to pull the data.<br><br>
  The selected time period (1985–2023) reflects the range of available historical data through <b>yfinance</b>. Finally, we renamed the default <b>'Close'</b> column to match the name of the corresponding index for consistency in the data.
</p>

In [7]:
# libraries
import yfinance as yf
import pandas as pd

# downloaded the data from yfinance
sp500_raw = yf.download('^GSPC', start = '1985-01-01', end = '2024-12-31', interval = '1mo', progress = False) # progress -- to supress warnings
nasdaq_raw = yf.download('^IXIC', start = '1985-01-01', end = '2024-12-31', interval = '1mo', progress = False)
dow_raw = yf.download('^DJI', start = '1985-01-01', end = '2024-12-31', interval = '1mo', progress = False)

# renamed column to match index name
sp500 = sp500_raw[['Close']].rename(columns={'Close': 'S&P 500'})
nasdaq = nasdaq_raw[['Close']].rename(columns={'Close': 'NASDAQ'})
dow = dow_raw[['Close']].rename(columns={'Close': 'Dow Jones'})
market_data = sp500.join([nasdaq, dow]).reset_index()

<p style="font-family: 'Times New Roman'; font-size: 18px; text-align: center; color: black;">
  Since we would eventually merge our stock market data with macroeconomic data from the Federal Reserve, which reports quarterly using report dates of January, April, July, and October, we decided to set our stock market data quarterly using these same months.
</p>

In [8]:
# consulted generative ai on April 19th, 2025 {prompt: How to convert monthly data into quarterly data using month indexes of 01,04,07,10}
market_data_quarterly = market_data[market_data['Date'].dt.month.isin([1, 4, 7, 10])].reset_index(drop = True)

<p style="font-family: 'Times New Roman'; font-size: 18px; text-align: center; color: black;">
  For our macroeconomic data from the Federal Reserve, we loaded it in using the standard <b>read_csv</b> from the <b>pandas</b> library. The factors we chose were CPI, Real Interest Rate, Unemployment Rate, Gross Domestic Product Growth, M2 Money Supply, and Federal Funds Rate. There are differences in how the data is reported, as CPI and GDP Growth are reported quarterly, Real Interest Rate, Unemployment Rate, and Federal Funds Rate are reported monthly, and M2 Money Supply is reported weekly. Therefore, we will need to convert all factors that are not reported quarterly to quarterly data with the month indexes of 01 (January), 04 (April), 07 (July), and 10 (October). Looking at the yearly data, we can start from January 1st, 1992 – October 10th, 2023.
</p>

In [9]:
cpi = pd.read_csv("CPI data.csv") # Jan 1, 1960 - Nov 1, 2023
ir = pd.read_csv("Real interest rate.csv") # Jan 1, 1982 - Mar 1, 2025
unemployment = pd.read_csv("Unemployment data.csv") # Jan 1, 1984 - Feb 1, 2025
gdp_growth = pd.read_csv("GDP growth rate.csv") # Apr 1, 1947 - Nov 1, 2024
m2_supply = pd.read_csv("M2 money supply.csv") # Nov 10, 1980 - Mar 3, 2025
fedfunds = pd.read_csv("FEDFUNDS.csv") # Jul 1, 1954 to March 1, 2023

<p style="font-family: 'Times New Roman'; font-size: 18px; text-align: center; color: black;">
  The cell below will work through the process of merging the data accordingly. In this code, we prepare and integrate multiple macroeconomic datasets to align them with quarterly stock market data for analysis. We begin by renaming columns across each dataset for clarity and consistency, particularly standardizing the date fields. We then convert all date columns to datetime format using pandas.to_datetime to facilitate accurate time-based operations. Since several macroeconomic indicators are reported at higher frequencies (monthly or weekly), we resample them to a quarterly frequency using the start of each quarter ('QS') and extract the first available observation to maintain consistency. To ensure a focused and relevant time frame, we filter all datasets to include only data from 1992 onward. Finally, we merge the datasets on the common Date column using inner joins, resulting in a merged quarterly dataset that includes CPI, real interest rates, unemployment, GDP growth, M2 money supply, federal funds rate, and market indices—providing a comprehensive foundation for subsequent analysis.
</p>

In [19]:
# Rename the columns to fit the stock market data
cpi = cpi.rename(columns = {'observation_date': 'Date', 'CPALTT01USQ661N': 'CPI'})
ir = ir.rename(columns = {'observation_date': 'Date', 'REAINTRATREARAT10Y': 'RealInterestRate'})
unemployment = unemployment.rename(columns = {'observation_date': 'Date', 'UNRATE': 'UnemploymentRate'})
gdp_growth = gdp_growth.rename(columns = {'observation_date': 'Date', 'A191RL1Q225SBEA': 'GDP_Growth'})
m2_supply = m2_supply.rename(columns = {'observation_date': 'Date', 'WM2NS': 'M2_Supply'})
fedfunds = fedfunds.rename(columns = {'observation_date': 'Date', 'FEDFUNDS': 'FedFundsRate'})

# Convert Date columns to fit the form of pd to_datime format
for date in [cpi, ir, unemployment, gdp_growth, m2_supply, fedfunds]:
    date['Date'] = pd.to_datetime(date['Date'])
market_data_quarterly['Date'] = pd.to_datetime(market_data_quarterly['Date'])

# For macroeconomic variables that are not quarterly, make them quarterly
# refrenced generative ai on April 19th, 2025 {prompt: How to convert monthly or weekly data into quarterly, output: set date column, resample using panda series QS, .first gets non-null first row, reset date index}
ir_q = ir.set_index('Date').resample('QS').first().reset_index()
unemp_q = unemployment.set_index('Date').resample('QS').first().reset_index()
m2_q = m2_supply.set_index('Date').resample('QS').first().reset_index()
fedfunds_q = fedfunds.set_index('Date').resample('QS').first().reset_index()

# Filter by date, 1992 and above
for date in [cpi, ir_q, unemp_q, gdp_growth, m2_q, fedfunds_q, market_data_quarterly]:
    date.reset_index(drop = True, inplace = True)
gdp_growth = gdp_growth[gdp_growth['Date'] >= '1992-01-01']
m2_q = m2_q[m2_q['Date'] >= '1992-01-01']
fedfunds_q = fedfunds_q[fedfunds_q['Date'] >= '1992-01-01']
market_data_quarterly = market_data_quarterly[market_data_quarterly['Date'] >= '1992-01-01']


# Merge all datasets on using the common column Date
merged_data = market_data_quarterly.merge(cpi, on = 'Date', how = 'inner')
merged_data = merged_data.merge(ir_q, on = 'Date', how = 'inner')
merged_data = merged_data.merge(unemp_q, on = 'Date', how = 'inner')
merged_data = merged_data.merge(gdp_growth, on = 'Date', how = 'inner')
merged_data = merged_data.merge(m2_q, on = 'Date', how = 'inner')
merged_data = merged_data.merge(fedfunds_q, on = 'Date', how = 'inner')

merged_data

,Date,S&P 500 ^GSPC,NASDAQ ^IXIC,Dow Jones ^DJI,CPALWE01USQ661N,RealInterestRate,UnemploymentRate,GDP_Growth,M2_Supply,FedFundsRate
0,1992-01-01,408.779999,620.210022,3223.399902,58.870052,3.198542,7.3,4.9,3398.6,4.03
1,1992-04-01,414.950012,578.679993,3359.100098,59.387717,3.789860,7.4,4.4,3434.1,3.73
2,1992-07-01,424.209991,580.830017,3393.800049,59.862244,3.353099,7.7,4.0,3407.1,3.25
3,1992-10-01,418.679993,605.169983,3226.300049,60.279252,2.728044,7.3,4.2,3413.8,3.10
4,1993-01-01,438.779999,696.340027,3310.000000,60.696260,3.267908,7.3,0.7,3455.2,3.02
...,...,...,...,...,...,...,...,...,...,...
123,2022-10-01,3871.979980,10988.150391,32732.949219,126.044174,1.797332,3.6,3.4,21429.5,3.08
124,2023-01-01,4076.600098,11584.549805,34086.039062,127.208058,1.783099,3.5,2.8,21364.4,4.33
125,2023-04-01,4169.479980,12226.580078,34098.160156,128.770113,1.443914,3.4,2.4,20924.5,4.83
126,2023-07-01,4588.959961,14346.019531,35559.531250,129.949384,1.426047,3.5,4.4,20744.8,5.12


<div style="font-family: 'Times New Roman'; font-size: 18px; text-align: center; color: black;">
  <p>The steps for merging our macroeconomic data with our stock market data are as follows:</p>
  <ol style="text-align: left; display: inline-block; margin-left: 0;">
    <li>
      Rename macroeconomic and date column names.<br>
      A. Used the <b>.rename</b> function with a dictionary to match <b>observation_date</b> to <b>Date</b>, and mapped each Fed variable name to a more descriptive label.
    </li>
    <li>
      Convert all date columns to datetime format.<br>
      B. Iterated through a list using a <b>for</b> loop and applied <b>pd.to_datetime()</b> to standardize date formats across datasets.
    </li>
    <li>
      Convert non-quarterly macroeconomic data to quarterly format.<br>
      C. Used <b>.resample('QS')</b> to align data with quarterly start months (Jan, Apr, Jul, Oct).
    </li>
    <li>
      Filter by date.<br>
      D. Retained only records from 1992 to 2023 by filtering each macroeconomic dataset.
    </li>
    <li>
      Merge all datasets by date.<br>
      E. Used the <b>.merge()</b> function on the <b>Date</b> column to integrate all sources into a single dataframe.
    </li>
  </ol>
</div>

In [20]:
merged_data.to_csv("merged_data.csv", index = False)

<p style="font-family: 'Times New Roman'; font-size: 18px; text-align: center; color: black;">
  For future use in notebooks, we saved it as a csv file. 
</p>

<div style="color: black; font-family: 'Times New Roman'; font-size: 18px;">
  <p style="text-align: center;"><strong>SOURCES</strong></p>
  
  <p style="text-align: center;">
    <a href="https://python-yahoofinance.readthedocs.io/en/latest/index.html" target="_blank" style="color: black; text-decoration: none;">
      <strong>Yfinance Data Documentation</strong>: https://python-yahoofinance.readthedocs.io/en/latest/index.html
    </a>
  </p>

  <p style="text-align: center;">
    <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html" target="_blank" style="color: black; text-decoration: none;">
      <strong>Pandas DataFrame Documentation</strong>: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html
    </a>
  </p>

  <p style="text-align: center;">
    <a href="https://fred.stlouisfed.org/" target="_blank" style="color: black; text-decoration: none;">
      <strong>Federal Reserve Data</strong>: https://fred.stlouisfed.org/
    </a>
  </p>
</div>